### Import required packages

In [ ]:
# Importing torch packages
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
from torchvision import datasets, transforms
from torchsummary import summary
import torchvision.models as models

# Importing other packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Creating a tensor
x = torch.tensor([[1, 2], [3, 4]])
print("Tensor x:\n", x)

In [ ]:
# Tensor operations
y = torch.ones(2, 2)
print("Tensor y:\n", y)
z = x + y
print("x + y:\n", z)

In [ ]:
# Autograd
# Requires_grad flag for automatic differentiation
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
y = torch.tensor([[5., 6.], [7., 8.]], requires_grad=True)
print(x)
print(y)

In [ ]:
# Computation
z = x + y
print("z:\n", z)

In [ ]:
# Computing gradients
t = torch.sum(z)
t.backward()

# Accessing gradients
print("Gradient of x:\n", x.grad)
print("Gradient of y:\n", y.grad)

# 2.Univariate Linear Regression

### Create the sample data

In [ ]:
# Generating a sample dataset
np.random.seed(0)
X_uni = np.random.rand(100, 1) * 10  # Years of experience (between 0 and 10)
y_uni = 3 * X_uni + 2 + np.random.randn(100, 1)  # Salary, with some noise

In [ ]:
# Visualizing the data
plt.scatter(X_uni, y_uni, label='Data')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.legend()
plt.title('Sample Dataset')
plt.show()

In [ ]:
# Convert the data to PyTorch tensors
X_uni_tensor = torch.tensor(X_uni, dtype=torch.float32)
y_uni_tensor = torch.tensor(y_uni, dtype=torch.float32)

In [ ]:
# Define the Linear Regression model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# Create an instance of the model
model = LinearRegressionModel()

In [ ]:
# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Training the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()  # Zero the gradients
    outputs = model(X_uni_tensor)  # Forward pass
    loss = criterion(outputs, y_uni_tensor)  # Compute the loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Making predictions
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_uni_tensor)

In [ ]:
# Convert predictions to numpy array
y_pred = y_pred_tensor.numpy()

In [ ]:
# Printing the model coefficients
print("Intercept:", model.linear.bias.item())
print("Coefficient:", model.linear.weight.item())

In [ ]:
# Plotting the results
plt.scatter(X_uni, y_uni, color='blue', label='Original data')
plt.plot(X_uni, y_pred, color='red', label='Fitted line')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.legend()
plt.show()


In [ ]:
# To test whether GPU instance is present in the system or not.
use_cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', use_cuda)

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")
device

# 3.Multivariate Classification

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=20, shuffle=True)

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=20, shuffle=True)

In [ ]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

## Plotting the images

In [ ]:
labels =[]
features = []
for X,y in zip(X_train, y_train):
  # Getting unique labels
  if y not in labels:
    labels.append(y)
    features.append(X)

pltsize=1
plt.figure(figsize=(7,7))
for i in range(5):
    plt.subplot(3,3, i+1)
    plt.axis('off')
    # Convert the tensor to numpy for displaying the image
    plt.imshow(features[i].numpy().reshape(28,28), cmap="gray")
    plt.title(f'Label: {labels[i]}')

### Dense Neural Network Classifiers

## Defining the Dense Neural Network’s Architecture

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = F.log_softmax(x, dim=1)
        return x

#### Calling the instances of the network

In [ ]:
model = Model()
model = model.to(device)
model

In [ ]:
summary(model, input_size=(1,28,28), batch_size=1)

#### Defining the loss function and optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

#### Training and Evaluating the model

In [ ]:
# No of Epochs
epoch = 2

# keeping the network in train mode
model.train()
train_losses,  train_accuracy = [], []

# Loop for no of epochs
for e in range(epoch):
    train_loss = 0
    correct = 0
    # Iterate through all the batches in each epoch
    for images, labels in train_loader:

      # Convert the image and label to gpu for faster execution
      images = images.to(device)
      labels = labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Passing the data to the model (Forward Pass)
      outputs = model(images)

      # Calculating the loss
      loss = criterion(outputs, labels)
      train_loss += loss.item()

      # Performing backward pass (Backpropagation)
      loss.backward()

      # optimizer.step() updates the weights accordingly
      optimizer.step()

      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()

    # Accuracy calculation
    train_losses.append(train_loss/len(mnist_trainset))
    train_accuracy.append(100 * correct/len(mnist_trainset))
    print('epoch: {}, Train Loss:{:.6f} Train Accuracy: {:.2f} '.format(e+1,train_losses[-1], train_accuracy[-1]))

In [ ]:
# Keeping the network in evaluation mode
model.eval()

Test_accuracy = 0

# Iterate through all the batches in each epoch
for images,labels in test_loader:

    # Convert the images and labels to gpu for faster execution
    images = images.to(device)
    labels = labels.to(device)

    # Do the forward pass
    outputs = model(images)

    # Accuracy calculation
    _, predicted = torch.max(outputs, 1)
    Test_accuracy += (predicted == labels).sum().item()

Accuracy = 100 * Test_accuracy / len(mnist_testset)
print("Accuracy of Test Data is", Accuracy)

## Transfer Learning with Pretrained Models

### Visualizing one image from the test dataset

In [ ]:
# Iterate through the testloader and extract one image and label
for images, labels in test_loader:
  image = images[0]  # Take the first image from the batch
  label = labels[0]  # Take the corresponding label
  break  # Exit the loop after extracting one sample

# Print the shape of the image and the label
print("Image shape:", image.shape)
print("Label:", label)

In [ ]:
# Reshape the image to [1, 1, 28, 28]
image_reshaped = image.unsqueeze(0)  # Add a batch dimension
print("Reshaped image shape:", image_reshaped.shape)

In [ ]:
# Convert the tensor to numpy for displaying the image
image_np = image_reshaped.squeeze().numpy()  # Remove batch dimension and convert to numpy

# Display the image
plt.imshow(image_np, cmap='gray')
plt.axis('off')  # Hide axes
plt.show()

### Load ResNet50 model and fine-tune it

In [ ]:
# Load pretrained ResNet50 model
model = models.resnet50(pretrained=True)

In [ ]:
# Modify the final fully connected layer to match the number of classes in Fashion MNIST
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Move the model to the device (GPU if available)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training is expected to take time

In [ ]:
# Train the model
epoch = 2
model.train()
train_losses, train_accuracy = [], []

for e in range(epoch):
    train_loss = 0
    correct = 0
    for images, labels in train_loader:
        resize_transform = transforms.Resize((224, 224))
        resized_image = resize_transform(images)
        resized_image = resized_image.repeat(1, 3, 1, 1)
        images = resized_image.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()

    train_losses.append(train_loss / len(mnist_trainset))
    train_accuracy.append(100 * correct / len(mnist_trainset))
    print('epoch: {}, Train Loss:{:.6f} Train Accuracy: {:.2f} '.format(e + 1, train_losses[-1], train_accuracy[-1]))

In [ ]:
# Evaluate the model
model.eval()
Test_accuracy = 0

for images, labels in test_loader:
    resize_transform = transforms.Resize((224, 224))
    resized_image = resize_transform(images)
    resized_image = resized_image.repeat(1, 3, 1, 1)
    images = resized_image.to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    Test_accuracy += (predicted == labels).sum().item()

Accuracy = 100 * Test_accuracy / len(mnist_testset)
print("Accuracy of Test Data is", Accuracy)